# Flat Fielding Mookodi Data
This code will take bias subtracted Mookodi fits files in the current directory, create a normalized master flat in each of the g and i frames and then divide each science image by the appropriate normalized master flat. 

In [ ]:

import os
from pathlib import Path
import glob
import numpy as np
from astropy.io import fits

## Create the master flat in a given filter
This function takes in a list of flat fields in a given filter and creates a normalized master flat.  Each input flat field is normalized by its mean value, and the master flat is created by taking a median of the individual normalized flat fields.  The code outputs the mean value and standard deviation of each individual flat field.  Exameine these values, and if any of them are off (means too high or low, or standard deviations odd), delete the associated individual flat field image and re-run the code (which will overwrite the previous master flat image). The master flat field is output as a fits file, with some keywords added to the header to indicate that we have created a measter flat field.

In [ ]:
###############################################################################

def make_master_flat (flat_files, filter, filename) : 
   flat_concat = []
   print("Creating flat in filter: ", filter)
   for i in flat_files:
       flat_hdulist = fits.open(i, ignore_missing_end=True)
       flat_hdu = flat_hdulist[0]
       flat_data = flat_hdu.data.astype(float)
       flat_mean = np.mean(flat_data)
       flat_std = np.std(flat_data)

       file_name = os.path.basename(i)
       print(file_name, filter, " mean: ", flat_mean, "stand. dev: ",flat_std)

#create normalized flats
       flat_concat.append(flat_data / flat_mean)

# take median of all biases to make master bias
   master_flat = np.median(flat_concat, axis=0)


#write out the master flat frame, adding keywords 
   new_hdu = flat_hdu
   new_hdu.header['MASTERFT'] = ("TRUE", "Master flat")
   new_hdu.header['NCOMBINE'] = (str(len(gflat_files)), "Median Combine")
   new_hdu.data = np.array((master_flat))
   new_hdu.writeto(filename, overwrite=True)
   flat_hdulist.close()
   print("Created master flat in filter", filter, " called ", filename)



## Ouput the flat fielded science images
This function take the list of sciene images in a given filter, and the associated master flat field and outputs the flat fielded science images.  An 'f' is pre-appended to the file name of the new fits files. 

In [ ]:
###############################################################################


def divide_flat (science_files, filter, master_flat) : 

   print()
   print("Flatfielding filter: ", filter)

   flat_hdulist = fits.open(master_flat, ignore_missing_end=True)
   flat_hdu = flat_hdulist[0]
   flat_data = flat_hdu.data.astype(float)

   for i in science_files:
       science_hdulist = fits.open(i, ignore_missing_end=True)
       science_hdu = science_hdulist[0]
       science_data = science_hdu.data.astype(float)

       file_name = "f" + os.path.basename(i)
       red_image = science_data/flat_data

# write out the flat fielded science images adding keywords 
       new_hdu = science_hdu
       new_hdu.header['FLATFIED'] = ("TRUE", "Divided by Flat Field")
       new_hdu.header['FLATNAME'] = (master_flat, "Name of flat field image")
       new_hdu.data = np.array((red_image))
       new_hdu.writeto(file_name, overwrite=True)
       flat_hdulist.close()
       print("Created flat field subtracted image:", file_name)


###############################################################################



## Main program
Loop over all of the bias subtracted fits files in a given directory and create a list of science image and flat field images in each of the g and i filters.

In [ ]:
my_cwd = Path(os.getcwd())
fits_files = list(my_cwd.glob('b*.fits'))

flat_keyword = "SKYFLAT"

iflat_files = []
gflat_files = []
gscience_files = []
iscience_files = []

for file in fits_files:
    hdul = fits.open(file)  # open a FITS file
    hdr = hdul[0].header  #extract the header information 

    if hdr['IMAGETYP'] == flat_keyword :
       if hdr['FILTERA'] == 'g' :
           gflat_files.append(file)
       elif hdr['FILTERA'] == 'i' :
           iflat_files.append(file)
       else :
           print("File is not a g or i filter: ", file,  hdr['FILTERA'] )
    else :
     #must be a science image
       if hdr['FILTERA'] == 'g' :
           gscience_files.append(file)
       elif hdr['FILTERA'] == 'i' :
           iscience_files.append(file)
       else :
           print("File is not a g or i filter: ",  file, hdr['FILTERA'])


if len(gflat_files) == 0:
       print("     !!! ERROR: no g flat  files found" )
       exit()
if len(iflat_files) == 0:
       print("     !!! ERROR: no i flat  files found" )
       exit()



## Create the master flat field in each filter

In [ ]:
make_master_flat (gflat_files, "g", "master_gflat.fits")
make_master_flat (iflat_files, "i", "master_iflat.fits")

## Divide the science images by the appropriate flat field and output the flat fielded images

In [ ]:
divide_flat(gscience_files, "g", "master_gflat.fits")
divide_flat(iscience_files, "i", "master_iflat.fits")
